In [31]:
#1
import pandas as pd

ratings = pd.read_csv("D:/Video/Нетология/Data_Science/Аналитика больших данных/Итоговая работа/ml-100k/u.data.csv", sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv("D:/Video/Нетология/Data_Science/Аналитика больших данных/Итоговая работа/ml-100k/u.item.csv", sep='|', header=None, names=['movie_id', 'movie_title', 'release_date', 'video_release', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')

In [2]:
#2
ratings.groupby('user_id').count().sort_values(by=['item_id'], ascending=False).reset_index().head(1)

,user_id,item_id,rating,timestamp
0,405,737,737,737


In [3]:
#3
ratings_user_one = ratings.loc[ratings['user_id']==405]

In [6]:
#4
ratings_movies = ratings.merge(movies, left_on='item_id', right_on='movie_id')

In [9]:
#5
from sklearn. model_selection import train_test_split
X = ratings_movies[['user_id', 'item_id', 'timestamp', 'movie_id', 'unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']]
y = ratings_movies['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 0)

In [20]:
#6
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
model.score(X_test, y_test)

0.06604259283379443

In [21]:
#7
from sklearn.metrics import accuracy_score
accuracy_score(y_test, lda.predict(X_test))

0.3545

In [36]:
#8
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

ratings_spark = spark.read.csv("D:/Video/Нетология/Data_Science/Аналитика больших данных/Итоговая работа/ml-100k/u.data.csv", sep='\t', header=None, inferSchema=True)
movies_spark = spark.read.csv("D:/Video/Нетология/Data_Science/Аналитика больших данных/Итоговая работа/ml-100k/u.item.csv", sep='|', header=None, inferSchema=True)

ratings_spark = ratings_spark.withColumnRenamed('_c0', 'user_id').withColumnRenamed('_c1', 'item_id').withColumnRenamed('_c2', 'rating').withColumnRenamed('_c3', 'timestamp')
movies_spark = movies_spark.withColumnRenamed('_c0', 'movie_id').withColumnRenamed('_c1', 'movie_title').withColumnRenamed('_c2', 'release_date').withColumnRenamed('_c3', 'video_release')\
    .withColumnRenamed('_c4', 'IMDb_URL').withColumnRenamed('_c5', 'unknown').withColumnRenamed('_c6', 'Action').withColumnRenamed('_c7', 'Adventure')\
    .withColumnRenamed('_c8', 'Animation').withColumnRenamed('_c9', 'Childrens').withColumnRenamed('_c10', 'Comedy').withColumnRenamed('_c11', 'Crime')\
    .withColumnRenamed('_c12', 'Documentary').withColumnRenamed('_c13', 'Drama').withColumnRenamed('_c14', 'Fantasy').withColumnRenamed('_c15', 'Film_Noir')\
    .withColumnRenamed('_c16', 'Horror').withColumnRenamed('_c17', 'Musical').withColumnRenamed('_c18', 'Mystery').withColumnRenamed('_c19', 'Romance')\
    .withColumnRenamed('_c20', 'Sci_Fi').withColumnRenamed('_c21', 'Thriller').withColumnRenamed('_c22', 'War').withColumnRenamed('_c23', 'Western')

In [76]:
#9
ratings_spark_avg = ratings_spark.groupBy('item_id').avg('rating')

ratings_movies_spark_avg = ratings_spark_avg.join(movies_spark, ratings_spark_avg.item_id == movies_spark.movie_id, how='inner')
ratings_movies_spark_avg['movie_title', 'avg(rating)'].show()

+--------------------+------------------+
|         movie_title|       avg(rating)|
+--------------------+------------------+
|It's a Wonderful ...| 4.121212121212121|
|Courage Under Fir...|3.6108597285067874|
|Secret of Roan In...| 3.859154929577465|
|Ghost and the Dar...|          3.203125|
|Convent, The (Con...|               2.5|
|  Bulletproof (1996)| 3.204081632653061|
|  Double Team (1997)| 2.230769230769231|
|Duoluo tianshi (1...|3.1666666666666665|
|   Full Speed (1996)|             3.125|
|     Liebelei (1933)|               1.0|
|Butcher Boy, The ...|               4.0|
|Man Without a Fac...|3.5441176470588234|
|Angels in the Out...| 2.923076923076923|
|  Money Train (1995)| 2.511627906976744|
|Amityville: Dollh...|               1.0|
|       Sirens (1994)| 2.983050847457627|
|Jungle2Jungle (1997)|2.4393939393939394|
|Fire Down Below (...|2.9318181818181817|
|Anne Frank Rememb...| 3.857142857142857|
|Truman Show, The ...| 2.909090909090909|
+--------------------+------------

In [102]:
#10 среднюю оценку для каждого жанра
mr = movies_spark.join(ratings_spark, ratings_spark.item_id == movies_spark.movie_id, how='inner')
mr = mr['movie_title', 'rating', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
# mr.where(mr['Action'] == 1).show()

mr.createOrReplaceTempView('m_r')
spark.sql('SELECT avg(rating) as avg_rating_unknown FROM m_r where unknown = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Action FROM m_r where Action = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Adventure FROM m_r where Adventure = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Animation FROM m_r where Animation = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Childrens FROM m_r where Childrens = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Comedy FROM m_r where Comedy = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Crime FROM m_r where Crime = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Documentary FROM m_r where Documentary = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Drama FROM m_r where Drama = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Fantasy FROM m_r where Fantasy = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Film_Noir FROM m_r where Film_Noir = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Horror FROM m_r where Horror = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Musical FROM m_r where Musical = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Mystery FROM m_r where Mystery = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Romance FROM m_r where Romance = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Sci_Fi FROM m_r where Sci_Fi = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Thriller FROM m_r where Thriller = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_War FROM m_r where War = 1').show()
spark.sql('SELECT avg(rating) as avg_rating_Western FROM m_r where Western = 1').show()


+------------------+
|avg_rating_unknown|
+------------------+
|               3.2|
+------------------+

+-----------------+
|avg_rating_Action|
+-----------------+
|3.480245417953027|
+-----------------+

+--------------------+
|avg_rating_Adventure|
+--------------------+
|   3.503526503308369|
+--------------------+

+--------------------+
|avg_rating_Animation|
+--------------------+
|  3.5766990291262135|
+--------------------+

+--------------------+
|avg_rating_Childrens|
+--------------------+
|  3.3532442216652742|
+--------------------+

+------------------+
| avg_rating_Comedy|
+------------------+
|3.3940734781442745|
+------------------+

+------------------+
|  avg_rating_Crime|
+------------------+
|3.6322780881440098|
+------------------+

+----------------------+
|avg_rating_Documentary|
+----------------------+
|    3.6728232189973613|
+----------------------+

+------------------+
|  avg_rating_Drama|
+------------------+
|3.6873793708484772|
+------------------+

+

In [69]:
#11
ratings_spark_max_5 = ratings_spark.groupBy('item_id').sum('rating')
ratings_spark_max_5 = ratings_spark_max_5.orderBy('sum(rating)', ascending=False)
ratings_spark_max_5.show(5)

ratings_spark_max_5 = ratings_spark.groupBy('item_id').sum('rating')
ratings_spark_min_5 = ratings_spark_max_5.orderBy('sum(rating)', ascending=True)
ratings_spark_min_5.show(5)

+-------+-----------+
|item_id|sum(rating)|
+-------+-----------+
|     50|       2541|
|    100|       2111|
|    181|       2032|
|    258|       1936|
|    174|       1786|
+-------+-----------+
only showing top 5 rows

+-------+-----------+
|item_id|sum(rating)|
+-------+-----------+
|    830|          1|
|   1339|          1|
|   1493|          1|
|   1621|          1|
|   1618|          1|
+-------+-----------+
only showing top 5 rows

